In [37]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.datasets import make_classification
from sklearn.neighbors import NearestNeighbors
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.mongoexp import MongoTrials
from types import SimpleNamespace

import os
from datetime import datetime
import sys
import time
try:
    from iterstrat.ml_stratifiers import MultilabelStratifiedKFold  # trainML
except:
    sys.path.append("../input/iterative-stratification")  # kaggle
    from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [24]:
BASE_PATH = os.environ.get('TRAINML_DATA_PATH') if os.environ.get('TRAINML_DATA_PATH') else '../input/lish-moa'
BASE_PATH

'/opt/trainml/input'

In [25]:
train_features = pd.read_csv(f'{BASE_PATH}/train_features.csv')
train_targets = pd.read_csv(f'{BASE_PATH}/train_targets_scored.csv')
test_features = pd.read_csv(f'{BASE_PATH}/test_features.csv')

sample_submission = pd.read_csv(f'{BASE_PATH}/sample_submission.csv')

### Data preprocessing

In [26]:
def preprocess(df):
    df = df.copy()
    df.loc[:, 'cp_type'] = df.loc[:, 'cp_type'].map({'trt_cp': 0, 'ctl_vehicle': 1})
    df.loc[:, 'cp_dose'] = df.loc[:, 'cp_dose'].map({'D1': 0, 'D2': 1})
    del df['sig_id']
    return df

train_data = preprocess(train_features)
test_data = preprocess(test_features)

del train_targets['sig_id']

train_targets = train_targets.loc[train_data['cp_type']==0].reset_index(drop=True)
train_data = train_data.loc[train_data['cp_type']==0].reset_index(drop=True)

In [27]:
X_original = train_data.values
Y_original = train_targets.values

X_test = test_data.values

### Data augmentation helper functions

Currently only doing data oversampling with MLSMOTE algorithm.

In [45]:
def get_tail_labels(df: pd.DataFrame, ql=[0.03, 1.]) -> list:
    " Find the underepresented targets a.k.a. minority labels. "
    irlbl = df.sum(axis=0)
    irlbl = irlbl[(irlbl > irlbl.quantile(ql[0])) & ((irlbl < irlbl.quantile(ql[1])))]  # Filtering
    irlbl = irlbl.max() / irlbl
    threshold_irlbl = irlbl.median()
    tail_labels = irlbl[irlbl > threshold_irlbl].index.tolist()
    return tail_labels

def get_minority_samples(X: pd.DataFrame, y: pd.DataFrame, ql=[0.03, 1.]):
    " Find minority samples associated with minority labels. "
    tail_labels = get_tail_labels(y, ql=ql)
    index = y[y[tail_labels].apply(lambda x: (x == 1).any(), axis=1)].index.tolist()
    
    X_sub = X[X.index.isin(index)].reset_index(drop = True)
    y_sub = y[y.index.isin(index)].reset_index(drop = True)
    return X_sub, y_sub

def nearest_neighbour(X: pd.DataFrame, neigh) -> list:
    " Find nearest neighbors for each sample in X dataframe. "
    nbs = NearestNeighbors(n_neighbors=neigh, metric='euclidean', algorithm='kd_tree').fit(X)
    euclidean, indices = nbs.kneighbors(X)
    return indices

def MLSMOTE(X, y, n_samples, n_neighbors=5):
    " Generate new samples using MLSMOTE algorithm. "
    indices2 = nearest_neighbour(X, neigh=n_neighbors)
    n = len(indices2)
    new_X = np.zeros((n_samples, X.shape[1]))
    target = np.zeros((n_samples, y.shape[1]))
    for i in range(n_samples):
        reference = random.randint(0, n-1)
        neighbor = random.choice(indices2[reference, 1:])
        all_point = indices2[reference]
        nn_df = y[y.index.isin(all_point)]
        ser = nn_df.sum(axis = 0, skipna = True)
        target[i] = np.array([1 if val > 0 else 0 for val in ser])
        ratio = random.random()
        gap = X.loc[reference,:] - X.loc[neighbor,:]
        new_X[i] = np.array(X.loc[reference,:] + ratio * gap)
    new_X = pd.DataFrame(new_X, columns=X.columns)
    target = pd.DataFrame(target, columns=y.columns)
    return new_X, target

def augment_data(X, y, oversample_args: tuple):
    " Augment feature/targets data (just doing oversampling for now)"
    n_samples, n_neighbors = oversample_args

    X_sub, y_sub = get_minority_samples(X, y)
    X_res, y_res = MLSMOTE(X_sub, y_sub, n_samples, n_neighbors)
    X_augmented = pd.concat([X, X_res])
    y_augmented = pd.concat([y, y_res])
    return X_augmented, y_augmented

### Define model + dataset

In [46]:
class MoaModel(nn.Module):
    def __init__(    
        self,
        n_columns,
        n_targets,
        layer1_outputs,
        layer1_dropout,
        layer2_outputs,
        layer2_dropout,
        layer3_enable,
        layer3_outputs,
        layer3_dropout,
        final_layer_dropout,
    ):
        super(MoaModel, self).__init__()   
        self.batch_norm1 = nn.BatchNorm1d(n_columns)
        self.dropout1 = nn.Dropout(layer1_dropout)
        self.dense1 = nn.utils.weight_norm(nn.Linear(n_columns, layer1_outputs))
        
        self.batch_norm2 = nn.BatchNorm1d(layer1_outputs)
        self.dropout2 = nn.Dropout(layer2_dropout)
        self.dense2 = nn.utils.weight_norm(nn.Linear(layer1_outputs, layer2_outputs))
        
        self.layer3 = layer3_enable
        if self.layer3:
            self.batch_norm3 = nn.BatchNorm1d(layer2_outputs)
            self.dropout3 = nn.Dropout(layer3_dropout)
            self.dense3 = nn.utils.weight_norm(
                nn.Linear(layer2_outputs, layer3_outputs)
            )
            
        final_layer_inputs = layer3_outputs if self.layer3 else layer2_outputs
        self.batch_norm_final = nn.BatchNorm1d(final_layer_inputs)
        self.dropout_final = nn.Dropout(final_layer_dropout)
        self.dense_final = nn.utils.weight_norm(nn.Linear(final_layer_inputs, n_targets))
        
    def forward(self, X):
        X = self.batch_norm1(X)
        X = self.dropout1(X)
        X = F.relu(self.dense1(X))
        
        X = self.batch_norm2(X)
        X = self.dropout2(X)
        X = F.relu(self.dense2(X))
        
        if self.layer3:
            X = self.batch_norm3(X)
            X = self.dropout3(X)
            X = F.relu(self.dense3(X))
            
        X = self.batch_norm_final(X)
        X = self.dropout_final(X)
        X = F.sigmoid(self.dense_final(X))
        
        return X
    
    def _load_from_file(self, file):
        self.load_state_dict(torch.load(file))
        
    def save(self, file):
        torch.save(self.state_dict(), file)
        
def batch_gd(model, device, criterion, optimizer, train_loader, val_loader, epochs):
    train_losses = np.zeros(epochs)
    val_losses = np.zeros(epochs)
    for it in range(epochs):
        t0 = datetime.now()

        model.train()
        train_loss = []
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, targets)

            loss.backward()
            optimizer.step()

            train_loss.append(loss.item() / len(train_loader))

        train_loss = np.mean(train_loss)

        model.eval()
        val_loss = []
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_loss.append(loss.item() / len(val_loader))
        val_loss = np.mean(val_loss)

        train_losses[it] = train_loss
        val_losses[it] = val_loss

        dt = datetime.now() - t0
        print(
            f"Epoch {it+1}/{epochs}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}, Duration {dt}"
        )

    return train_losses, val_losses

In [47]:
class MoaDataset(Dataset):
    def __init__(self, features, targets, mode="train"):
        self.mode = mode
        self.data = features
        if mode == "train":
            self.targets = targets

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if self.mode == "train":
            return torch.FloatTensor(self.data[idx]), torch.FloatTensor(
                self.targets[idx]
            )
        elif self.mode == "eval":
            return torch.FloatTensor(self.data[idx]), 0

### Helper functions for hyperopt's `objective`

In [48]:
def augment_data(X, y, n_samples, n_neighbors):
    """
    Augment feature/targets data with oversampling (using MLSMOTE algorithm)
    Parameters for hyperopt search space:
        n_samples - # of data points to add for underrepresented labels
        n_neighbors - used for MLSMOTE algorithm.
    """
    X_sub, y_sub = get_minority_samples(X, y)
    X_res, y_res = MLSMOTE(X_sub, y_sub, n_samples, n_neighbors)
    X_augmented = pd.concat([X, X_res])
    y_augmented = pd.concat([y, y_res])
    return X_augmented, y_augmented

# Lines to replicate in hyperopt:
# train_data_augmented, train_targets_augmented = augment_data(train_data, train_targets, 1000, 5)
# X = train_data_augmented.values
# Y = train_targets_augmented.values

def make_model(
        n_columns,
        n_targets,
        layer1_outputs,
        layer1_dropout,
        layer2_outputs,
        layer2_dropout,
        layer3_enable,
        layer3_outputs,
        layer3_dropout,
        final_layer_dropout,
        device
    ):
    """
    Make a 2 or 3 layer neural network with specified outputs and dropout per layer.
    These 8 function parameters are exposed for hyperopt search space.
    """
    model = MoaModel(n_columns, n_targets, layer1_outputs, layer1_dropout, layer2_outputs,
                   layer2_dropout, layer3_enable, layer3_outputs, layer3_dropout, final_layer_dropout)
    model.to(device)
    return model

# Lines to replicate in hyperopt:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = make_model(n_columns=train_data.shape[1], <....>, n_targets=206, device)

def train(model, device, X, Y, n_splits, batch_size, epochs):
    """
    Run model training. Batch size, # splits for k-fold iteration, and number of epochs
    are exposed for hyperopt search space.
    Returns tuple of (train_losses, val_losses).
    """
    kfold = MultilabelStratifiedKFold(n_splits=n_splits, random_state=42, shuffle=True)

    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters()) # TODO parameterize?

    train_losses = np.array([])
    val_losses = np.array([])

    for n, (tr, te) in enumerate(kfold.split(X, Y)):
        X_train, X_val = X[tr], X[te]
        y_train, y_val = Y[tr], Y[te]

        train_dataset = MoaDataset(X_train, y_train)
        val_dataset = MoaDataset(X_val, y_val)
        train_loader = torch.utils.data.DataLoader(
            dataset=train_dataset, batch_size=batch_size, shuffle=True
        )
        val_loader = torch.utils.data.DataLoader(
            dataset=val_dataset, batch_size=batch_size, shuffle=False
        )
        split_train_losses, split_val_losses = batch_gd(
            model, device, criterion, optimizer, train_loader, val_loader, epochs
        )
        print(
            f"Fold {n+1}, final train loss: {split_train_losses[epochs-1]:5.5f}, final train loss: {split_val_losses[epochs-1]:5.5f}"
        )
        train_losses = np.concatenate((train_losses, split_train_losses))
        val_losses = np.concatenate((val_losses, split_val_losses))

    model.save("latest_model")
    return train_losses, val_losses

# Lines to replicate in hyperopt:
# train_losses, val_losses = train_model(model, device, X, Y, n_splits=10, batch_size=4096, epochs=50)

### Hyperopt config - objective function and search space

In [51]:
def objective(space):
    args = SimpleNamespace(**space)
    
    # Augment data
    train_data, train_targets = augment_data(args.train_data, args.train_targets, args.n_samples, args.n_neighbors)
    X = train_data.values
    Y = train_targets.values
    
    # Build model architecture
    device = args.device
    model = make_model(
                args.n_columns,
                args.n_targets,
                args.layer1_outputs,
                args.layer1_dropout,
                args.layer2_outputs,
                args.layer2_dropout,
                args.layer3_enable,
                args.layer3_outputs,
                args.layer3_dropout,
                args.final_layer_dropout,
                args.device
    )
    
    # Train model
    train_losses, val_losses = train(model, device, X, Y, args.n_splits, args.batch_size, args.epochs)

    return {
        "status": STATUS_OK,
        "model_result": model,
        "train_losses": train_losses,
        "val_losses": val_losses
    }
    

space = {
    # general
    'device': torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    'train_data': train_data,
    'train_targets': train_targets,
    
    # data augmentation
    'n_samples': hp.randint('n_samples', 5000),
    'n_neighbors': 3 + hp.randint('n_neighbors', 10),
    
    # model architecture
    # TODO: add optimizer
    'n_columns': train_data.shape[1],
    'n_targets': train_targets.shape[1],
    'layer1_outputs': 256 + hp.randint('layer1_outputs', 4096),
    'layer1_dropout': hp.uniform('layer1_dropout', 0, 1),
    'layer2_outputs': 256 + hp.randint('layer2_outputs', 4096),
    'layer2_dropout': hp.uniform('layer2_dropout', 0, 1),
    'layer3_enable': hp.choice('layer3_enable', [True, False]),
    'layer3_outputs': 256 + hp.randint('layer3_outputs', 4096),
    'layer3_dropout': hp.uniform('layer3_dropout', 0, 1),
    'final_layer_dropout': hp.uniform('final_layer_dropout', 0, 1),
    
    # cross-validation
    'n_splits': 5 + hp.randint('n_splits', 10),
    'batch_size': 1 + hp.randint('batch_size', 8192),
    'epochs': 1 + hp.randint('epochs', 100),
    'eval_time': time.time()
    
}

### Run model training via hyperopt

In [52]:
trials = Trials() # TODO: Make it MongoTrials
best = fmin(
    objective,
    space=space,
    algo=tpe.suggest,
    max_evals=100,
    trials=trials
    # max_queue_len=4 <-- what again is this used for? multiple workers?
)

  0%|          | 0/100 [00:00<?, ?trial/s, best loss=?]

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass shuffle=True, random_state=42 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)

/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")



Epoch 1/2, Train Loss: 0.0895, Validation Loss: 0.3522, Duration 0:00:00.845679
Epoch 2/2, Train Loss: 0.0861, Validation Loss: 0.3357, Duration 0:00:00.682713
Fold 1, final train loss: 0.08613, final train loss: 0.33573
Epoch 1/2, Train Loss: 0.0832, Validation Loss: 0.3117, Duration 0:00:00.656299
Epoch 2/2, Train Loss: 0.0775, Validation Loss: 0.2802, Duration 0:00:00.680086
Fold 2, final train loss: 0.07755, final train loss: 0.28018
Epoch 1/2, Train Loss: 0.0687, Validation Loss: 0.2363, Duration 0:00:00.570218
Epoch 2/2, Train Loss: 0.0576, Validation Loss: 0.2014, Duration 0:00:00.693558
Fold 3, final train loss: 0.05764, final train loss: 0.20136
Epoch 1/2, Train Loss: 0.0468, Validation Loss: 0.1623, Duration 0:00:00.671936
Epoch 2/2, Train Loss: 0.0378, Validation Loss: 0.1266, Duration 0:00:00.650479
Fold 4, final train loss: 0.03777, final train loss: 0.12663
Epoch 1/2, Train Loss: 0.0308, Validation Loss: 0.1052, Duration 0:00:00.588255
Epoch 2/2, Train Loss: 0.0255, Valid

job exception: 



  0%|          | 0/100 [00:22<?, ?trial/s, best loss=?]


InvalidLoss: 

In [ ]:
print(best)

In [ ]:
plt.plot(train_losses, label='train loss')
plt.plot(val_losses, label='validation loss')
plt.legend()
plt.show()

### Generate predictions

In [2]:
def predict(model, device, data_loader):
    model.eval()
    preds = []

    for inputs, _ in data_loader:
        inputs = inputs.to(device)

        with torch.no_grad():
            outputs = model(inputs)

        preds.append(outputs.detach().cpu().numpy())

    preds = np.concatenate(preds)

    return preds

test_dataset = MoaDataset(X_test, None, mode='eval')
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)
preds = predict(model, device, test_loader)

NameError: name 'MoaDataset' is not defined

In [ ]:
targets = [col for col in train_targets.columns]
sample_submission[targets] = preds
sample_submission.loc[test_features['cp_type']=='ctl_vehicle', targets] = 0
sample_submission.to_csv('submission.csv', index=False)

### Model evaluation

In [ ]:
train_dataset = MoaDataset(X_original, None, mode='eval')
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=False)
mock_preds = predict(model, device, train_loader)

In [ ]:
loss_fn = nn.BCELoss()

y_pred = torch.from_numpy(mock_preds.astype(float))
y_true = torch.from_numpy(Y_original.astype(float))

score = loss_fn(y_pred, y_true)
print("log-loss score: ", score.item())

In [ ]:
from sklearn.metrics import roc_curve, auc, roc_auc_score

# compute ROC AUC curve and AUC score using micro-averaging
fpr, tpr, _ = roc_curve(y_true.ravel(), y_pred.ravel())
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='darkorange',
         lw=2, label='ROC curve (area = %0.4f)' % roc_auc)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Aggregate ROC curve (micro-averaging)')
plt.legend(loc="lower right")
plt.show()

print("ROC AUC score: ", roc_auc_score(y_true, y_pred, average='micro'))

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix

decision_boundary = 0.5
confusion_matrices = multilabel_confusion_matrix(y_true, (y_pred > decision_boundary))

agg_confusion_matrix = np.array([[0, 0],  # [tn, fp]
                                 [0, 0]])  # [fn, tp]
for m in confusion_matrices:
    agg_confusion_matrix += m

print(agg_confusion_matrix)